In [1]:
import pandas as pd
import numpy as np
from joblib import Parallel, delayed

from utility_functions import get_acc, record
from BIC import BIC
from MLP import mlp_evaluate

In [2]:
# dataset
dataset = 'detailed'

# training data
fold_path = 'training_data/' + dataset + '/folds.csv'
inputs_path = 'training_data/' + dataset + '/inputs.csv'
outputs_path = 'training_data/' + dataset + '/outputs.csv'
evaluation_path = 'training_data/' + dataset + '/evaluation.csv'

# writing accuracy rate path
acc_rate_path = 'acc_rate/' + dataset + '.csv'

# path to write df to csv
output_df_path = 'record_dataframe/' + dataset + '/'

# raw dfs
fold_df = pd.read_csv(fold_path)
inputs_df = pd.read_csv(inputs_path)
outputs_df = pd.read_csv(outputs_path)
evaluation_df = pd.read_csv(evaluation_path)

# feature engineering transformation
identity = lambda x: x
log      = lambda x: np.log(x)
loglog   = lambda x: np.log(np.log(x))

In [3]:
# raw dfs
fold = 1

# fold dfs
fold_inputs_df = inputs_df[inputs_df['sequenceID'].isin(fold_df[fold_df['fold'] == fold]['sequenceID'])]
fold_outputs_df = outputs_df[outputs_df['sequenceID'].isin(fold_df[fold_df['fold'] == fold]['sequenceID'])]
fold_eval_df = evaluation_df[evaluation_df['sequenceID'].isin(fold_df[fold_df['fold'] == fold]['sequenceID'])]

In [4]:
# BIC
lldas_test_df_BIC = BIC(fold_inputs_df)
print(get_acc(fold_eval_df, lldas_test_df_BIC))

85.10928961748634


In [5]:
# HYPER
n_layer = 0
layer_size = 1
chosen_feature = ['length']
f_engineer = [loglog]
normalize = 0

lldas_test_df_linear = mlp_evaluate(
    input_train_df = fold_inputs_df,
    output_train_df = fold_outputs_df,
    inputs_val_df = fold_inputs_df,
    hidden_layers = n_layer,
    hidden_size = layer_size,
    chosen_feature = chosen_feature,
    f_engineer = f_engineer,
    normalize = normalize
)
print(get_acc(fold_eval_df, lldas_test_df_linear))

92.62295081967213


In [9]:
# HYPER
n_layer = 0
layer_size = 1
chosen_feature = ['length', 'sd']
f_engineer = [log, log]
normalize = 0

lldas_test_df_linear = mlp_evaluate(
    input_train_df = fold_inputs_df,
    output_train_df = fold_outputs_df,
    inputs_val_df = fold_inputs_df,
    hidden_layers = n_layer,
    hidden_size = layer_size,
    chosen_feature = chosen_feature,
    f_engineer = f_engineer,
    normalize = normalize
)
print(get_acc(fold_eval_df, lldas_test_df_linear))

95.08196721311475


In [10]:
# HYPER
n_layer = 0
layer_size = 1
chosen_feature = ['length', 'sd']
f_engineer = [loglog, log]
normalize = 0

lldas_test_df_linear = mlp_evaluate(
    input_train_df = fold_inputs_df,
    output_train_df = fold_outputs_df,
    inputs_val_df = fold_inputs_df,
    hidden_layers = n_layer,
    hidden_size = layer_size,
    chosen_feature = chosen_feature,
    f_engineer = f_engineer,
    normalize = normalize
)
print(get_acc(fold_eval_df, lldas_test_df_linear))

95.08196721311475


In [7]:
# HYPER
n_layer = 1
layer_size = 8
chosen_feature = ['sd', 'range_value', 'length', 'sum_diff']
f_engineer = [log, log, loglog, log]
normalize = 1

lldas_test_df_linear = mlp_evaluate(
    input_train_df = fold_inputs_df,
    output_train_df = fold_outputs_df,
    inputs_val_df = fold_inputs_df,
    hidden_layers = n_layer,
    hidden_size = layer_size,
    chosen_feature = chosen_feature,
    f_engineer = f_engineer,
    normalize = normalize
)
print(get_acc(fold_eval_df, lldas_test_df_linear))

96.58469945355192
